In [1]:
%reload_ext autoreload
%autoreload 2

import os, sys
sys.path.append(os.environ['DH_DIR'])
sys.path.append('..')

import numpy as np
import pickle
import time
from tqdm import tqdm

from low_energy.lowE_electrons import make_interpolator
from low_energy.lowE_deposition import compute_fs

from config import load_data
from main import get_elec_cooling_data
from   darkhistory.spec.spectrum import Spectrum
import darkhistory.physics as phys
import darkhistory.spec.spectools as spectools
from   darkhistory.electrons import positronium as pos
from   darkhistory.electrons.elec_cooling import get_elec_cooling_tf

## Config

In [32]:
run_name = '230408x'
tf_type = 'phot' # {'phot', 'elec'}
include_f_xray = True
xray_eng_range = (1e3, 1e4) # [eV]
use_tqdm = True
verbose = 0 # {0, 1}
stop_after_n = np.inf

abscs = pickle.load(open(f'../data/abscissas/abscs_{run_name}.p', 'rb'))
inj_abscs = abscs['photE'] if tf_type == 'phot' else abscs['elecEk'] + phys.me
MEDEA_dir = '../data/MEDEA'
data_dir = f'../data/tf/230408/{tf_type}'
save_dir = f'../data/tf/{run_name}/{tf_type}'
os.makedirs(save_dir, exist_ok=True)

In [5]:
abscs['dep_c']

['H ion', 'He ion', 'exc', 'heat', 'cont', 'xray']

## Load

In [3]:
print('Loading tf: ', end=' ', flush=True)
hep_tfgv = np.load(f'{data_dir}/hep_tf_rxneo.npy')
print('hep', end=' ', flush=True)
lep_tfgv = np.load(f'{data_dir}/lep_tf_rxneo.npy')
print('lep', end=' ', flush=True)
lee_tfgv = np.load(f'{data_dir}/lee_tf_rxneo.npy')
print('lee', end=' ', flush=True)
hed_tfgv = np.load(f'{data_dir}/hed_tf_rxneo.npy')
print('hed', end=' ', flush=True)
cmbloss_gv = np.load(f'{data_dir}/cmbloss_rxneo.npy')
print('cmb', end='.', flush=True)

Loading tf:  hep lep lee hed cmb.

## Loop over `rs` `x` `nBs`

In [33]:
tfgv = np.zeros_like(hep_tfgv) # in: elec/phot, out: phot
depgv = np.zeros(
    hed_tfgv.shape[:-1] + (len(abscs['dep_c']),)
) # channels: {H ionization, He ionization, excitation, heat, continuum}

dlnz = abscs['dlnz']
MEDEA_interp = make_interpolator(prefix=MEDEA_dir)

n_run = -1

if use_tqdm:
    pbar = tqdm( total = len(abscs['rs'])*len(abscs['x'])*len(abscs['nBs']) )

for i_rs, rs in enumerate(abscs['rs']):
    
    dt = dlnz / phys.hubble(rs)
            
    #==============================
    # Add cmbloss to highengphot
    cmb_un = spectools.discretize(abscs['photE'], phys.CMB_spec, phys.TCMB(rs))
    cmb_un_E = cmb_un.toteng()
    
    for i_x, x in enumerate(abscs['x']):
        for i_nBs, nBs in enumerate(abscs['nBs']):
            
            for i in range(len(inj_abscs)):
                cmb_E = cmbloss_gv[i_rs, i_x, i_nBs][i] * dt
                hep_tfgv[i_rs, i_x, i_nBs][i] += (-cmb_E/cmb_un_E) * cmb_un.N

            #==============================
            # Add lowengphot diagonal
            if nBs == 0: # lowengphot is 0 when nBs is 0
                raise NotImplementedError
            if tf_type == 'phot':
                for i in range(len(inj_abscs)):
                    if lep_tfgv[i_rs, i_x, i_nBs][i][i] > 1e-40:
                        break
                    lep_tfgv[i_rs, i_x, i_nBs][i][i] = 1
            
            for i_injE, injE in enumerate(inj_abscs):
                
                assert n_run <= stop_after_n
                n_run += 1

                #==============================
                # Injection
                # inject one photon at i_injE
                timer = time.time()
                
                hep_spec_N = hep_tfgv[i_rs, i_x, i_nBs, i_injE]
                lep_spec_N = lep_tfgv[i_rs, i_x, i_nBs, i_injE]
                lee_spec_N = lee_tfgv[i_rs, i_x, i_nBs, i_injE]
                hed_arr    = hed_tfgv[i_rs, i_x, i_nBs, i_injE]

                lowengelec_spec_at_rs = Spectrum(abscs['elecEk'], lee_spec_N, spec_type='N')
                lowengelec_spec_at_rs.rs = rs

                lowengphot_spec_at_rs = Spectrum(abscs['photE'], lep_spec_N, spec_type='N')
                lowengphot_spec_at_rs.rs = rs

                highengdep_at_rs = hed_arr

                #==============================
                # Compute f's
                x_vec_for_f = np.array( [1-x, phys.chi*(1-x), x] ) # [HI, HeI, HeII]/nH
                nBs_ref = 1
                dE_dVdt_inj = injE * phys.nB * nBs_ref * rs**3 / dt # [eV/cm^3 s]
                # in DH.main: (dN_inj/dB) / (dE_inj  /dVdt)
                # here:       (dN_inj   ) / (dE_injdB/dVdt)
                f_low, f_high = compute_fs(
                    MEDEA_interp=MEDEA_interp,
                    rs=rs,
                    x=x_vec_for_f,
                    elec_spec=lowengelec_spec_at_rs,
                    phot_spec=lowengphot_spec_at_rs,
                    dE_dVdt_inj=dE_dVdt_inj,
                    dt=dt,
                    highengdep=highengdep_at_rs,
                    cmbloss=0, # turned off in darkhistory main as well
                    method='no_He',
                    cross_check=False,
                    ion_old=False
                )
                f_raw = f_low + f_high

                #==============================
                # Compute tf & f values
                lep_prop_spec_N = lep_spec_N * (abscs['photE'] < 10.2)
                f_lep_prop = np.dot(abscs['photE'], lep_prop_spec_N) / injE
                phot_spec_N = hep_spec_N + lep_prop_spec_N
                f_prop = np.dot(abscs['photE'], phot_spec_N) / injE

                f_dep = f_raw
                f_dep[4] -= f_lep_prop # adjust for the propagating lowengphot
                f_tot = f_prop + np.sum(f_dep)

                #==============================
                # Fix energy conservation (known issues)
                if tf_type == 'phot':
                    if i_injE == 153: # issue at around 13.6 eV. Adjusting H_ion.
                        f_dep[0] += 1 - f_tot
                    if i_injE in range(224, 228): # ??? issue. Adjusting hep propagating bin.
                        phot_spec_N[i_injE] += 1 - f_tot

                f_prop = np.dot(abscs['photE'], phot_spec_N) / injE
                f_tot = f_prop + np.sum(f_dep)

                #==============================
                # Energy conservation
                f_dep_str = ' '.join([f'{v:.3e}' for v in f_dep])
                print_str = f'{n_run} | {i_rs} {i_x} {i_nBs} {i_injE} | f_prop={f_prop:.6f} f_dep={f_dep_str} f_tot={f_tot:.6f}'
                if np.abs(f_tot - 1.) > 1e-2:
                    print_str += ' | Energy error > 1%'
                if verbose >= 1 or np.abs(f_tot - 1.) > 1e-2:
                    print(print_str, flush=True)
                
                phot_spec_N[i_injE] += 1 - f_tot # gives all extra energy to propagating photons
                
                #==============================
                # Dependent variables (Xray)
                if include_f_xray:
                    i_xray_fm = np.searchsorted(abscs['photE'], xray_eng_range[0])
                    i_xray_to = np.searchsorted(abscs['photE'], xray_eng_range[1])
                    f_xray = np.dot(abscs['photE'][i_xray_fm:i_xray_to], phot_spec_N[i_xray_fm:i_xray_to]) / injE
                    f_dep = np.append(f_dep, f_xray)

                #==============================
                # Populate transfer functions
                tfgv[i_rs, i_x, i_nBs, i_injE] = phot_spec_N
                depgv[i_rs, i_x, i_nBs, i_injE] = f_dep
            
            if use_tqdm:
                pbar.update()

 79%|███████▉  | 789/1000 [15:25<04:09,  1.18s/it]

394730 | 7 8 9 230 | f_prop=0.872475 f_dep=1.362e-04 9.247e-06 1.143e-04 1.172e-01 6.561e-06 f_tot=0.989904 | Energy error > 1%


 80%|███████▉  | 799/1000 [15:36<03:55,  1.17s/it]

399730 | 7 9 9 230 | f_prop=0.872400 f_dep=1.320e-04 6.717e-06 1.120e-04 1.172e-01 6.459e-06 f_tot=0.989897 | Energy error > 1%


 87%|████████▋ | 869/1000 [16:58<02:33,  1.17s/it]

...

100%|█████████▉| 998/1000 [19:29<00:02,  1.17s/it]

499230 | 9 9 8 230 | f_prop=0.867067 f_dep=1.406e-04 7.010e-06 1.193e-04 1.223e-01 6.123e-06 f_tot=0.989622 | Energy error > 1%


100%|█████████▉| 999/1000 [19:30<00:01,  1.17s/it]

499729 | 9 9 9 229 | f_prop=0.646992 f_dep=3.909e-04 2.236e-05 3.312e-04 3.652e-01 2.115e-05 f_tot=1.012917 | Energy error > 1%
499730 | 9 9 9 230 | f_prop=0.725872 f_dep=2.955e-04 1.444e-05 2.506e-04 2.519e-01 1.351e-05 f_tot=0.978375 | Energy error > 1%
499731 | 9 9 9 231 | f_prop=0.786760 f_dep=2.509e-04 1.075e-05 2.131e-04 1.990e-01 9.953e-06 f_tot=0.986269 | Energy error > 1%
499733 | 9 9 9 233 | f_prop=0.869763 f_dep=1.693e-04 5.776e-06 1.444e-04 1.195e-01 5.217e-06 f_tot=0.989547 | Energy error > 1%


100%|██████████| 1000/1000 [19:31<00:00,  1.17s/it]

## Save

In [ ]:
np.save(f'{save_dir}/{tf_type}_tfgv.npy', tfgv)
np.save(f'{save_dir}/{tf_type}_depgv.npy', depgv)